# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pickle
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from flask import jsonify
import re
import ast
import requests

In [2]:
# LOADING ESSENTIAL DATASETS
collaborative = pd.read_csv("collaborative.csv")
data = pd.read_csv("required.csv")

In [3]:
# LOADING MODELS
COLL = pickle.load(open('model/COLL.sav', 'rb'))
CONT = pickle.load(open('model/CONT.sav', 'rb'))

In [4]:
def data_and_recommend(tmdb, movie_id):
    
    # COll
    tmdb_coll = []
    if (movie_id != None) and (not pd.isna(movie_id)):
        # FIND NEAREST MOVIES
        _, indices = COLL.kneighbors(collaborative[collaborative['movieId'] == movie_id].drop('movieId', axis=1).values.reshape(1, -1), n_neighbors = 12)
        movieId = collaborative.iloc[indices[0]]['movieId']
        tmdb_coll = data[data['movie_id'].isin(movieId)]['tmdb_id']
        #coll = data[data['movie_id'].isin(movieId)]['movie_title']
    
    if len(tmdb_coll) == 0:
        l = 12
    else:
        l = 6
    
    # CONT
    # INDICES
    indices = pd.Series(data.index, index=data['tmdb_id'].apply(lambda x: int(x))).drop_duplicates()
    rev = pd.Series(data['tmdb_id'].apply(lambda x: int(x)), index=data.index).drop_duplicates()
    # MOVIE INDEX
    idx = indices[tmdb]
    # GET PAIRWISE SIMILARITY SCORE 
    sig_scores = list(enumerate(CONT[idx]))
    # SORTING MOVIES
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    # GET MOST SIMILAR MOVIES PAIRS
    sig_scores = [ i[0] for i in sig_scores[1:1+l]]
    # GET TMDB ID
    tmdb_con = []
    for i in sig_scores:
        try:
            tmdb_con.append(rev[i])
        except:
            pass
    # PRINT SIMILAR MOVIES
    # con = [data[data['tmdb_id'] == float(i)]['movie_title'].values[0] for i in tmdb_con]
    
    #REMOVE SIMILIAR MOVIES BETWEEN coll AND cont
    tmdb_coll = list(set(tmdb_coll) - set(tmdb_con))[:6]
    
    # MAKING THE DATA TO BE RENDERED IN HTML
    m = data[data['tmdb_id'] == tmdb]
    
    URL = "https://api.themoviedb.org/3/movie/" +str(m['tmdb_id'].values.tolist()[0])+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
    r = requests.get(url = URL).json()

    dict = {}
    
    # DETAILS OF THE CURRENT MOVIE
    dict['movie_title'] = m['movie_title'].values.tolist()[0]
    try:
        dict['overview'] = r['overview']
    except:
        dict['overview'] = ' '
    try:
        dict['vote_average'] = r['vote_average']
    except:
        dict['vote_average'] = ' '
    try:
        dict['vote_count'] = r['vote_count']
    except:
        dict['vote_count'] = ' '
    try:
        dict['genres'] = ", ".join([i['name'] for i in r['genres']])
    except:
        dict['genres'] = ' '
    try:
        dict['release_date'] = r['release_date']
    except:
        dict['release_date'] = ' '
    try:
        dict['runtime'] = r['runtime']
    except:
        dict['runtime'] = ' '
    try:
        dict['status'] = r['status']
    except:
        dict['status'] = ' '
    try:
        dict['poster_path'] = "https://image.tmdb.org/t/p/w154" + r['poster_path']
    except:
        dict['poster_path'] = 'https://www.publicdomainpictures.net/pictures/280000/velka/not-found-image-15383864787lu.jpg'
    try:
        dict['backdrop_path'] = "https://image.tmdb.org/t/p/original" + r['backdrop_path']
    except:
        dict['backdrop_path'] = 'https://whatnerd.com/wp-content/uploads/2021/04/sites-rent-movies-online-fandangonow-featured.jpg'
    
    # DETAILS OF THE CAST
    cast = []
    URL = "https://api.themoviedb.org/3/movie/" +str(m['tmdb_id'].values.tolist()[0])+ "/credits?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
    r = requests.get(url = URL).json()
    for i in r['cast'][:12]:
        d = {}
        URL = "https://api.themoviedb.org/3/person/" +str(i['id'])+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
        b = requests.get(url = URL).json()
        try:
            d['name'] = i['name']
        except:
            d['name'] = ' '
        try:
            d['character'] = i['character']
        except:
            d['character'] = ' '
        try:
            d['profile_path'] = "https://image.tmdb.org/t/p/w154" + i['profile_path']
        except:
            d['profile_path'] = 'https://www.publicdomainpictures.net/pictures/280000/velka/not-found-image-15383864787lu.jpg'
        try:
            d['biography'] = b['biography']
        except:
            d['biography'] = ' '
        cast.append(d)
    dict['cast'] = cast
    
    # CONTENT BASED RECOMMENDATION
    CON_M = []
    for i in tmdb_con:
        con_m = data[data['tmdb_id'] == i]
        d = {}
        d['movie_title'] = con_m['movie_title'].values.tolist()[0]
        URL = "https://api.themoviedb.org/3/movie/" +str(i)+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
        r = requests.get(url = URL).json()
        try:
            d['poster_path'] = "https://image.tmdb.org/t/p/w154" + r['poster_path']
        except:
            d['poster_path'] = 'https://www.publicdomainpictures.net/pictures/280000/velka/not-found-image-15383864787lu.jpg'
        CON_M.append(d)
    dict['CON_M'] = CON_M
    
    # COLLABORATION BASED RECOMMENDATION
    COLL_M = []
    for i in tmdb_coll:
        coll_m = data[data['tmdb_id'] == i]
        d = {}
        d['movie_title'] = coll_m['movie_title'].values.tolist()[0]
        URL = "https://api.themoviedb.org/3/movie/" +str(i)+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
        r = requests.get(url = URL).json()
        try:
            d['poster_path'] = "https://image.tmdb.org/t/p/w154" + r['poster_path']
        except:
            d['poster_path'] = 'https://www.publicdomainpictures.net/pictures/280000/velka/not-found-image-15383864787lu.jpg'
        COLL_M.append(d)
    dict['COLL_M'] = COLL_M
    
    return dict

In [ ]:
data_and_recommend(19995, 72998)

# FLASK

In [18]:
app = Flask(__name__)
run_with_ngrok(app)

# LOADING PRETRAINED MODEL
@app.route("/")
def home():
    return render_template("index.html")


@app.route('/suggest', methods=['GET','POST'])
def suggest():
    if request.method == "POST":
        x = request.data.decode("utf-8")
        x = data[data["movie_title"].str.contains(x, flags=re.IGNORECASE)]["movie_title"].values.tolist()
        return jsonify(x)
    
@app.route("/recommend", methods=["POST"])
def recommend():
    movie = request.form['txt']
    row = data[data["movie_title"].str.contains(movie, flags=re.IGNORECASE)]
    if len(row) != 0:
        ex = data[data['tmdb_id'] == row['tmdb_id'].values[0]][['tmdb_id', 'movie_id']].values[0].tolist()
        result = data_and_recommend(ex[0], ex[1])
        return render_template("result.html",result = result)
    else:
        return render_template("index.html", text="Movie not found, Please select from dropdown",)
@app.route("/link_recommend", methods=["GET"])
def link_recommend():
    movie = request.args.get('name', default = '*', type = str)
    row = data[data["movie_title"].str.contains(movie, flags=re.IGNORECASE)]
    ex = data[data['tmdb_id'] == row['tmdb_id'].values[0]][['tmdb_id', 'movie_id']].values[0].tolist()
    result = data_and_recommend(ex[0], ex[1])
    return render_template("result.html",result = result)
    
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Nov/2021 00:03:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:03:47] "POST /suggest HTTP/1.1" 200 -


 * Running on http://d862-103-226-237-28.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Nov/2021 00:03:51] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:03:59] "POST /suggest HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:04:08] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:05:20] "POST /suggest HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:05:22] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:05:38] "POST /suggest HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:05:50] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:06:14] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:06:28] "POST /suggest HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:06:44] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:06:58] "POST /suggest HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:07:07] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 00:07:21] "GET /link_recommend?name=Get%20Hard HTTP/1.1" 200 -
127.0.0.1 - - [23/No